In [ ]:
# 匯入需要的函式庫
from transformers import BertTokenizerFast, GPT2LMHeadModel
import torch

# 使用中研院的 GPT2 中文模型
model_name = "ckiplab/gpt2-base-chinese"

# 載入 tokenizer（用來把文字轉換為模型可處理的數字 ID）
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# 載入 GPT2 中文模型
model = GPT2LMHeadModel.from_pretrained(model_name)

# 為了讓 GPT2 模型能正常處理 padding，設定 pad_token 為 eos_token（結束符號）
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# 初始輸入文字，由使用者輸入
text = input("請輸入文字:")

num_return_sequences = 3       # 每次產生 3 個選項
max_length = 20                # 每個候選最多 20 個字元

# 持續進行輸入與選擇的迴圈，直到使用者輸入 "exit"
while True:
    # 將目前的文字轉為模型可讀格式（張量）
    inputs = tokenizer(text, return_tensors="pt")

    # 使用 GPT2 模型產生多個續句（採樣方式可讓內容多樣）
    outputs = model.generate(
        inputs["input_ids"],              # 模型輸入張量
        max_new_tokens=max_length,       # 最大續句長度
        num_return_sequences=num_return_sequences,  # 要產生幾個句子
        do_sample=True,                  # 啟用隨機採樣
        top_k=40,                        # 保留機率前 40 的詞
        top_p=0.92,                      # nucleus sampling（只考慮總機率達 0.92 的詞）
        temperature=0.9,                 # 控制輸出隨機性
        repetition_penalty=1.1,          # 避免一直說同樣的話
        no_repeat_ngram_size=2,          # 禁止 2-gram 重複
        early_stopping=False,            # 不提早停止產生
        pad_token_id=tokenizer.pad_token_id  # 設定 pad_token，防止錯誤
    )

    # 取得目前輸入文字的最後一個字，用來後續避免接續句首重複
    last_char = text.strip()[-1] if text.strip() else ""

    # 過濾重複與無效選項，整理候選續句
    filtered_options = []
    for output in outputs:
        # 將模型輸出從 token ID 轉為文字
        decoded = tokenizer.decode(output, skip_special_tokens=True)

        # 取出生成內容中「新增加的部分」
        continuation = decoded[len(text):].strip()

        # 如果接續句開頭重複了原句的最後一個字，則移除該字以避免語意不順
        if continuation.startswith(last_char):
            continuation = continuation[1:].lstrip()

        # 去除空字串與重複句子，才加入候選清單
        if continuation and continuation not in filtered_options:
            filtered_options.append(continuation)

        # 一旦候選數量達到需求就停止
        if len(filtered_options) >= num_return_sequences:
            break

    # 顯示所有候選句供使用者選擇
    print("\n請選擇以下接續句：")
    for i, option in enumerate(filtered_options):
        print(f"{i+1}: {option}")

    # 取得使用者的選擇
    choice = input("\n請輸入選項編號（或輸入 exit 離開）：")

    # 如果使用者輸入 exit，就結束程式
    if choice.lower() == "exit":
        print("感謝使用，再見！")
        break

    # 檢查輸入是否有效（是數字且在可選範圍內）
    if not choice.isdigit() or not (1 <= int(choice) <= len(filtered_options)):
        print("無效選項，請重新選擇。")
        continue

    # 取得使用者選定的句子，並加到原始輸入上（用空格隔開）
    selected = filtered_options[int(choice) - 1]
    text += " " + selected

    # 顯示目前累積的完整句子
    print(f"\n當前句子：{text}")


請輸入文字:我在


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.



請選擇以下接續句：
1: 國 的 政 治 生 涯, 最 後 也 能 獲 得 民 主 黨 內 初 選 。
2: 世 紀 已 經 出 現 一 些 特 殊 的 事 情 」, 但 他 相 信 自
3: 場 上 一 直 是 我 的 驕 傲, 這 也 許 我 認 為 我 是 真 正

請輸入選項編號（或輸入 exit 離開）：3


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.



當前句子：我在 場 上 一 直 是 我 的 驕 傲, 這 也 許 我 認 為 我 是 真 正

請選擇以下接續句：
1: 愛 台 灣 。 」 「 我 對 我 們 國 內 政 治 、 經 濟 與 社 會
2: 傲 慢 的 女 人 。 」 另 外 還 有 三 名 台 灣 人 和 兩 名 日
3: 個 能 夠 代 表 我 國 民 眾 的 國 家 元 首 。 他 相 信 今 後

請輸入選項編號（或輸入 exit 離開）：3


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.



當前句子：我在 場 上 一 直 是 我 的 驕 傲, 這 也 許 我 認 為 我 是 真 正 個 能 夠 代 表 我 國 民 眾 的 國 家 元 首 。 他 相 信 今 後

請選擇以下接續句：
1: 我 會 繼 續 為 兩 岸 三 通 做 努 力, 以 加 速 兩 地 人 民
2: 個 世 紀 將 會 更 加 努 力 。 我 今 天 應 邀 前 往 澳 洲 參
3: 將 會 與 各 界 人 士 見 面, 彼 此 更 重 要 的 目 標 就 是

請輸入選項編號（或輸入 exit 離開）：1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.



當前句子：我在 場 上 一 直 是 我 的 驕 傲, 這 也 許 我 認 為 我 是 真 正 個 能 夠 代 表 我 國 民 眾 的 國 家 元 首 。 他 相 信 今 後 我 會 繼 續 為 兩 岸 三 通 做 努 力, 以 加 速 兩 地 人 民

請選擇以下接續句：
1: 流 與 經 濟 往 來, 對 中 共 政 權 及 台 灣 當 局 發 展 有
2: 福 祉 與 合 作 。 」 900328 民 主 進 步 黨 籍 立 法 委 員
3: 往 來 、 增 進 中 華 文 化 與 其 他 方 面 的 發 展 。 蕭 萬
